In [9]:
import string
import re
import pandas as pd
import os
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from nltk.stem import PorterStemmer
from keras.models import load_model

# load doc into memory
def load_doc(filename):
    # open the file as read only
    
    file = open(filename, errors="ignore")
    #file = open(filename, 'r')
    
    # read all text
    text = file.read()
    
    
    
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
    
def clean_doc(doc, vocab):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

# load all docs in a directory
def process_docs(directory, vocab, is_train):
    documents = list()
        
    # walk through all files in the folder
    for filename in listdir(directory):
        
        
        # skip any reviews in the test set
        if is_train and filename.startswith('p0'):  ##Check filename
            continue
            
        if not is_train and filename.startswith('trn'):  #check filename
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
               
        # load the doc
        doc = load_doc(path)
        
        """
        if directory is 'neg':
            #os.chdir('C:/Users/ejl9900/Desktop/Data Science/MSDS453/NLP Project - Survey Results/neg')
            #doc = pd.read_csv('Survey Results Negative.csv', encoding='cp1252')
            doc = load_doc ('C:/Users/ejl9900/Desktop/Data Science/MSDS453/NLP Project - Survey Results/neg/Survey Results Negative - Copy.csv')
        
        if directory is 'pos':
            #os.chdir('C:/Users/ejl9900/Desktop/Data Science/MSDS453/NLP Project - Survey Results/pos')
            #doc = pd.read_csv('Survey Results Positive.csv', encoding='cp1252')
            doc = load_doc ('C:/Users/ejl9900/Desktop/Data Science/MSDS453/NLP Project - Survey Results/pos/Survey Results Positive v2.csv')
        
        """
        
        # clean doc
        tokens = clean_doc(doc, vocab)
        # add to list
        documents.append(tokens)
  
    return documents

# load and clean a dataset
def load_clean_dataset(vocab, is_train):
    # load documents
    #neg = process_docs('txt_sentoken/neg', vocab, is_train)
    #pos = process_docs('txt_sentoken/pos', vocab, is_train)
    neg = process_docs('neg', vocab, is_train)
    pos = process_docs('pos', vocab, is_train)
    
    
    docs = neg + pos

    # prepare labels
    labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
    
    #print(labels)
    
    return docs, labels

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
    # integer encode
    encoded = tokenizer.texts_to_sequences(docs)
    # pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

# define the model
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    #model.add(Conv1D(32, 8, activation='relu'))  #8
    model.add(Conv1D(32, 20, activation='relu'))  #8
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))  #10
    model.add(Dense(1, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  #optimizer='adam'
    # summarize defined model
    model.summary()
    plot_model(model, to_file='model_survey.png', show_shapes=True)
    return model


# load the vocabulary
vocab_filename = 'vocab_survey.txt'
os.chdir('C:/Users/ejl9900/Desktop/Data Science/MSDS453/NLP Project - Survey Results')
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())

# load training data
train_docs, ytrain = load_clean_dataset(vocab, True)

#print(train_docs)

# create the tokenizer
tokenizer = create_tokenizer(train_docs)


# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
#15.5. Evaluate Model 167
print('Vocabulary size: %d' % vocab_size)

# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])  #######
print('Maximum length: %d' % max_length)

# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)

# define model
model = define_model(vocab_size, max_length)

# fit network
model.fit(Xtrain, ytrain, epochs=15, verbose=2)

# save the model
model.save('model_survey.h5')



###Evaluation Model###


# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, max_length, model):
    # clean review
    line = clean_doc(review, vocab)
    # encode and pad review
    padded = encode_docs(tokenizer, max_length, [line])
    # predict sentiment
    yhat = model.predict(padded, verbose=0)
    # retrieve predicted percentage and label
    percent_pos = yhat[0,0]
    if round(percent_pos) == 0:
        return (1-percent_pos), 'NEGATIVE'
    return percent_pos, 'POSITIVE'

# load the vocabulary
vocab_filename = 'vocab_survey.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())

# load all reviews
train_docs, ytrain = load_clean_dataset(vocab, True)
test_docs, ytest = load_clean_dataset(vocab, False)

# create the tokenizer
tokenizer = create_tokenizer(train_docs)

# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)

# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)

# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
Xtest = encode_docs(tokenizer, max_length, test_docs)


# load the model
model = load_model('model_survey.h5')

# evaluate model on training dataset
_, acc = model.evaluate(Xtrain, ytrain, verbose=0)
print('Train Accuracy: %.2f' % (acc*100))

# evaluate model on test dataset
_, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %.2f' % (acc*100))


# test positive text
#text1 = 'I love this class and recommend it. The discussion was very valuable and I felt very comfortable.  The instructor did a great job and I love this training'
#text1 = 'This training was great. Cindi was fantastic.  I learned alot.'
text1 = 'I really liked the unconscious Bias material and exercises.  It was a great class and fantastic. I would recommend'
percent, sentiment = predict_sentiment(text1, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text1, sentiment, percent*100))

# test negative text
#text2 = 'This was a waste of time. It was horrible and boring.  I felt very uncomfortable.  The training was too long.'
#text2 = 'I felt very uncomfortable. It was too long and horrible. The examples are overused, boring, and were poorly incorporated into the training.'
text2 = 'I felt very uncomfortable. It was very glitchy and too long.'
percent, sentiment = predict_sentiment(text2, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text2, sentiment, percent*100))




Vocabulary size: 1612
Maximum length: 2683
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 2683, 100)         161200    
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 2664, 32)          64032     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 1332, 32)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 42624)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                426250    
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 651,493
Trainable params: 651,493
Non-trainable params: 0
_____